In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/A-LLMRec-20241107 2nd for colab for api

In [ ]:
!pip install -r "/content/drive/MyDrive/Colab Notebooks/A-LLMRec-20241107 2nd for colab for api/requirements.txt"

In [ ]:
# .env 파일에서 환경 변수 로드
from dotenv import load_dotenv
import os

load_dotenv()

import threading
import time
import psutil
import torch
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Dict
import asyncio  # asyncio 모듈 임포트
from concurrent.futures import ThreadPoolExecutor 

# 모델 코드에서 필요한 모듈 임포트
from chat_bot import Chat_bot
import nest_asyncio
import uvicorn
from pyngrok import ngrok

# .env 파일에서 환경 변수 가져오기
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
NGROK_AUTH_TOKEN = os.getenv("NGROK_AUTH_TOKEN")

# 환경 변수 설정
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["NGROK_AUTH_TOKEN"] = NGROK_AUTH_TOKEN

# 공통 모델 설정 (필요 시 수정)
args = type('Args', (), {})()
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.multi_gpu = False
args.rec_pre_trained_data = 'All_Beauty'
args.recsys = 'sasrec'
args.pretrain_stage1 = False
args.pretrain_stage2 = False
args.inference = True
args.llm = 'opt'
args.maxlen = 50

# FastAPI 앱 생성
app = FastAPI()

executor = ThreadPoolExecutor(max_workers=2)

chatbots: Dict[int, Chat_bot] = {}

class GenerateRequest(BaseModel):
    user_id: int
    prompt: str

class UserRequest(BaseModel):
    user_id: int

@app.post("/start_instance")
async def start_instance(request: UserRequest):
    user_id = request.user_id
    if user_id in chatbots:
        return {"message": f"User {user_id}'s model is already running."}
    else:
        # Chat_bot 인스턴스 생성
        new_chatbot = Chat_bot(args, user_id=user_id)
        chatbots[user_id] = new_chatbot
        return {"message": f"User {user_id}'s model has been started."}

@app.post("/stop_instance")
async def stop_instance(request: UserRequest):
    user_id = request.user_id
    if user_id in chatbots:
        # 인스턴스 자원 해제
        chatbot = chatbots[user_id]
        # 필요한 경우 자원 해제 
        del chatbots[user_id]
        return {"message": f"User {user_id}'s model has been stopped."}
    else:
        return {"message": f"User {user_id}'s model is not running."}

@app.post("/generate")
async def generate_text(request: GenerateRequest):
    user_id = request.user_id
    prompt = request.prompt

    if user_id in chatbots:
        chatbot = chatbots[user_id]
        loop = asyncio.get_running_loop()
        with torch.no_grad():
            # 동기 함수를 스레드에서 비동기로
            response = await loop.run_in_executor(executor, chatbot.handle_request, prompt)
        # 필요에 따라 torch.cuda.empty_cache() 
        return {"generated_text": response}
    else:
        return {"generated_text": "모델이 켜져있지 않습니다."}

# GPU 및 메모리 사용량 모니터링
stop_event = threading.Event()

def monitor_resource_usage():
    while not stop_event.is_set():
        gpu_memory_allocated = torch.cuda.memory_allocated() / (1024 ** 3)
        gpu_memory_reserved = torch.cuda.memory_reserved() / (1024 ** 3)
        cpu_memory = psutil.virtual_memory()
        cpu_memory_used = (cpu_memory.total - cpu_memory.available) / (1024 ** 3)
        cpu_memory_total = cpu_memory.total / (1024 ** 3)

        # 메모리 사용량 출력 (GPU 사용량이 특정 임계치 이상일 경우만)
        if gpu_memory_allocated > 1.0:  # 예: 1GB 이상일 때만 출력
            print(f"GPU 메모리 사용량: {gpu_memory_allocated:.2f} GB / {gpu_memory_reserved:.2f} GB (할당 / 예약)")
            print(f"CPU 메모리 사용량: {cpu_memory_used:.2f} GB / {cpu_memory_total:.2f} GB")

        time.sleep(20)  # 출력 주기를 20초로 설정
    print("모니터링 스레드가 종료되었습니다.")

monitor_thread = threading.Thread(target=monitor_resource_usage, daemon=True)
monitor_thread.start()

# 서버 시작 시 0번 사용자 인스턴스 자동 생성
print("0번 사용자 인스턴스 자동 생성")
chatbots[0] = Chat_bot(args, user_id=0)

# 이벤트 루프 관련 설정 (필요한 경우에만 사용)
nest_asyncio.apply()

# Uvicorn 서버 실행
def run_app():
    config = uvicorn.Config(
        app,
        host="0.0.0.0",
        port=8001,
        log_level="info",
        loop="asyncio",  # uvloop 대신 asyncio 이벤트 루프 사용
        workers=2  # workers를 2로 설정
    )
    server = uvicorn.Server(config)
    server.run()

run_app()  # 서버 실행

# 메인 스레드 유지 및 종료 시 리소스 정리
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("프로그램이 중지되었습니다.")
    stop_event.set()  # 모니터링 스레드 종료 설정
    monitor_thread.join()  # 모니터링 스레드 종료 대기
    ngrok.kill()  # ngrok 터널 종료
    print("모든 스레드와 ngrok 터널이 종료되었습니다.")


In [1]:
from pyngrok import ngrok
ngrok.kill()